# Notebook setup

In [1]:
import numpy as np
import os
import onnx
from onnxruntime_extensions import get_library_path, PyOp, onnx_op, PyOrtFunction
import onnxruntime as ort
from qonnx.core.datatype import DataType
from qonnx.core.modelwrapper import ModelWrapper
from onnx.helper import make_attribute
from driver_base import FINNExampleOverlay
from driver import io_shape_dict
set_pyop = False

platform = "zynq-iodma"
batch_size = 1
bitfile = "finn-accel.bit"
outputfile = "output.npy"
runtime_weight_dir = "runtime_weights/"

# instantiate FINN accelerator driver and pass batchsize and bitfile
accel = FINNExampleOverlay(
    bitfile_name = bitfile, platform = platform,
    io_shape_dict = io_shape_dict, batch_size = batch_size,
    runtime_weight_dir = runtime_weight_dir
)

if not set_pyop:
    # Implement the CustomOp by decorating a function with onnx_op
    @onnx_op(op_type="StreamingDataflowPartition", inputs=[PyOp.dt_float], outputs=[PyOp.dt_float])
    def StreamingDataflowPartition(inputs):
        obuf_normal = accel.execute(inputs)
        return obuf_normal.astype(np.float32)
    set_pyop = True
print(set_pyop)

True


In [5]:
from qonnx.transformation.double_to_single_float import DoubleToSingleFloat
from onnx import helper

def set_multithreshold_default(model):#,save_model):
    '''
    Pass a modelproto model and the save file
    '''
    model = model.transform(DoubleToSingleFloat())
    new_attr = [helper.make_attribute("out_scale", 1.0),
                helper.make_attribute("out_bias", 0.0),
                helper.make_attribute("data_layout","NCHW")]

    for n in model.graph.node:
        if n.op_type == "MultiThreshold":
            out_scale,bias,datalayout = False,False,False
            for na in n.attribute:
                if na.name == "out_scale": out_scale = True
                if na.name == "out_bias": bias = True
                if na.name == "data_layout": datlayout = True
            if not out_scale: n.attribute.append(new_attr[0])
            if not bias: n.attribute.append(new_attr[1])
            if not datalayout: n.attribute.append(new_attr[2])

            n.domain = "ai.onnx.contrib"
#     model.save(save_model)
    return model

def revert_quantAvgPool(model):
    nodes = [n for n in model.graph.node if n.op_type == 'QuantAvgPool2d']
    attrs = [n.attribute for n in model.graph.node if n.op_type == 'QuantAvgPool2d']
    for node,attr in zip(nodes,attrs):
        for a in attr:
            if a.name == "stride":
                s = a.i
            elif a.name == "kernel":
                k = a.i
        update = helper.make_node(
            "AveragePool",
            inputs=[node.input[0]],
            outputs=[node.output[0]],
            kernel_shape=[k,k],
            strides=[s,s],
        )

        model.graph.node.remove(node)
        model.graph.node.append(update)
    return model

## HW session for partition_0

In [3]:
inp = np.load("dog.npy")

model_func = PyOrtFunction.from_model("./dataflow_parent_updated.onnx")
outputs = model_func(inp)

## Continue with the output and pass to the ORT instance

In [7]:
model = ModelWrapper("./partition_1.onnx")

so = ort.SessionOptions()
so.register_custom_ops_library(get_library_path())

model = revert_quantAvgPool(model)
model = set_multithreshold_default(model)
sess = ort.InferenceSession(model.model.SerializeToString(),so)

inp_name = [i.name for i in sess.get_inputs()]
inp_shape = [i.shape for i in sess.get_inputs()]

print(inp_shape,outputs.shape)

# res = sess.run([],{inp_name[0]:outputs})
# y = np.array(res)
# y.shape

2022-12-16 09:25:43.599345820 [W:onnxruntime:, graph.cc:107 MergeShapeInfo] Error merging shape info for output. 'QuantAvgPool2d_0_out0' source:{1,7,1,1018} target:{1,1,1,1024}. Falling back to lenient merge.
2022-12-16 09:25:43.795555139 [E:onnxruntime:, sequential_executor.cc:369 Execute] Non-zero status code returned while running MatMul node. Name:'MatMul_0' Status Message: matmul_helper.h:61 Compute MatMul dimension mismatch


[[1, 512, 14, 14]] (1, 512, 14, 14)


Fail: [ONNXRuntimeError] : 1 : FAIL : Non-zero status code returned while running MatMul node. Name:'MatMul_0' Status Message: matmul_helper.h:61 Compute MatMul dimension mismatch

## Compare the two outputs

In [40]:
equal = np.testing.assert_array_almost_equal(y[0],outputs)
if equal == None:
    print("CPU and HW output are the same!")

CPU and HW output are the same!
